In [1]:
import pandas as pd
import pyarrow.parquet as pq
from sklearn.model_selection import GridSearchCV, train_test_split

# НЕ проверял работспособность, на этапе вычисления шанса выпадения встрял, очень долго это происходило

In [2]:
LOCAL_DATA_PATH = './context_data/'
SPLIT_SEED = 42
DATA_FILE = 'competition_data_final_pqt'
TARGET_FILE = 'public_train.pqt'
SUBMISSION_FILE = 'submit_2.pqt'
id_to_submit = pq.read_table(f'{LOCAL_DATA_PATH}/{SUBMISSION_FILE}')
data = pq.read_table(f'{LOCAL_DATA_PATH}/{DATA_FILE}').to_pandas()

In [3]:
#Классификация возрастов
df = pd.read_parquet(f'{LOCAL_DATA_PATH}/public_train.pqt')
df.loc[df['age'] > 66, 'age'] = 6
df.loc[(df['age'] > 55) & (df['age'] < 66), 'age'] = 5
df.loc[(df['age'] > 45) & (df['age'] < 56), 'age'] = 4
df.loc[(df['age'] > 35) & (df['age'] < 46), 'age'] = 3
df.loc[(df['age'] > 25) & (df['age'] < 36), 'age'] = 2
df.loc[(df['age'] > 18) & (df['age'] < 26), 'age'] = 1

In [4]:
#чтения файла с урлами и объединение этого с id-шниками
prev_df = data
df = pd.merge(df, prev_df, on='user_id')

In [ ]:
#one_hot_url_df = pd.get_dummies(df['url_host'])
#Жрет много памяти, очень много

In [6]:
df

,age,is_male,user_id,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,date,part_of_day,request_cnt
0,2.0,1,350459,Удмуртская Республика,Ижевск,Xiaomi,Redmi Note 7,sun9-29.userapi.com,smartphone,Android,14948.0,2021-06-24,morning,1
1,2.0,1,350459,Удмуртская Республика,Ижевск,Xiaomi,Redmi Note 7,id.tinkoff.ru,smartphone,Android,14948.0,2021-07-14,morning,1
2,2.0,1,350459,Удмуртская Республика,Ижевск,Xiaomi,Redmi Note 7,id.tinkoff.ru,smartphone,Android,14948.0,2021-07-13,evening,1
3,2.0,1,350459,Удмуртская Республика,Ижевск,Xiaomi,Redmi Note 7,vk.com,smartphone,Android,14948.0,2021-06-23,day,1
4,2.0,1,350459,Удмуртская Республика,Ижевск,Xiaomi,Redmi Note 7,vk.com,smartphone,Android,14948.0,2021-07-11,day,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210730727,6.0,1,103148,Белгородская область,Губкин,Samsung,Galaxy M21,googleads.g.doubleclick.net,smartphone,Android,14990.0,2021-06-28,day,1
210730728,6.0,1,103148,Курская область,Курск,Samsung,Galaxy M21,yandex.ru,smartphone,Android,14990.0,2021-07-08,morning,2
210730729,6.0,1,103148,Курская область,Курск,Samsung,Galaxy M21,i.ytimg.com,smartphone,Android,14990.0,2021-07-13,day,1
210730730,6.0,1,103148,Курская область,Курск,Samsung,Galaxy M21,news.ngs.ru,smartphone,Android,14990.0,2021-07-06,day,1


In [ ]:
# Подсчет шансов
age_list= list(range(1,7))
url_list = df['url_host'].unique()


url_stat_dict = {
    'url':[],
    'total_count':[],
    'fem': [],
    'male': []

}
for age in age_list:
    url_stat_dict['age_class_№_' + str(age)] = []

for key in url_list:
    url_stat_dict['url'].append(key)
    total_count = df[df['url_host'] == key].count()
    url_stat_dict['total_count'].append(total_count)
    url_stat_dict['fem'].append(df[df['url_host'] == key][df['is_male'] == 0].count()/total_count )
    url_stat_dict['male'].append(df[df['url_host'] == key][df['is_male'] == 1].count() / total_count)
    for age in age_list:
        url_stat_dict['age_class_№_' + str(age)].append(df[df['url_host'] == key][df['age'] == age].count() / total_count)

stat_df = pd.DataFrame().from_dict(url_stat_dict)

df = pd.merge(df, stat_df, on = 'url_host')

# Добавил столбцы 
# total_count - сумарное количество записей
# male/fem - шнас того, что этот сайт посетила мужчина/женщина
# age_class_№_* - шнас того, что этот сайт посетила человек определенного возраста

/tmp/ipykernel_15428/1214924861.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  url_stat_dict['fem'].append(df[df['url_host'] == key][df['is_male'] == 0].count()/total_count )
/tmp/ipykernel_15428/1214924861.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  url_stat_dict['male'].append(df[df['url_host'] == key][df['is_male'] == 1].count() / total_count)
/tmp/ipykernel_15428/1214924861.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  url_stat_dict['age_class_№_' + str(age)].append(df[df['url_host'] == key][df['age'] == age].count() / total_count)


In [ ]:
# Все модели обучаются только на прогно возраста, поменять можно заменив в строке с y age на is_male
# Линейная регрессия
from sklearn.linear_model import LinearRegression

X = df.drop(["age", 'is_male'], axis=1)
y = df["age"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

reg = LinearRegression()
param_grid = {'fit_intercept': [True, False], 'normalize': [True, False]}
grid_search = GridSearchCV(reg, param_grid, cv=5)
grid_search.fit(X_train, y_train)

test_score = grid_search.score(X_test, y_test)
print("Test Score:", test_score)

best_params = grid_search.best_params_
print("Best Params:", best_params)

In [ ]:
# Гребневая регрессия
from sklearn.linear_model import Ridge

reg = Ridge()

param_grid = {'alpha': [0.1, 1, 10, 100],
              'fit_intercept': [True, False],
              'normalize': [True, False]}

grid_search = GridSearchCV(reg, param_grid, cv=5)
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Test score:", grid_search.score(X_test, y_test))

In [ ]:
# Случайный лес
from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor()
param_grid = {'n_estimators': [10, 50, 100, 200], 'max_depth': [None, 10, 50, 100]}
grid_search = GridSearchCV(reg, param_grid, cv=5)

grid_search.fit(X_train, y_train)

test_score = grid_search.score(X_test, y_test)
print("Test Score:", test_score)

best_params = grid_search.best_params_
print("Best Params:", best_params)

In [ ]:
# Опорные вектора
from sklearn.svm import SVC

svm = SVC()

param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001]}
grid_search = GridSearchCV(svm, param_grid, cv=5)
grid_search.fit(X_train, y_train)

test_score = grid_search.score(X_test, y_test)
print("Test Score:", test_score)

best_params = grid_search.best_params_
print("Best Params:", best_params)

In [ ]:
# Градиентный бустинг
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor()
param_grid = {'n_estimators': [100, 500, 1000], 'max_depth': [1, 3, 5]}
grid_search = GridSearchCV(gbr, param_grid, cv=5)
grid_search.fit(X_train, y_train)

test_score = grid_search.score(X_test, y_test)
print("Test Score:", test_score)

best_params = grid_search.best_params_
print("Best Params:", best_params)

In [ ]:
# KNN
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
param_grid = {'n_neighbors': [3, 5, 7, 10], 'weights': ['uniform', 'distance']}
grid_search = GridSearchCV(knn, param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best Params:", best_params)

test_score = grid_search.score(X_test, y_test)
print("Test Score:", test_score)